<a href="https://colab.research.google.com/github/Agentoma/mmai5400A_assignment3/blob/main/assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipywidgets --upgrade
!pip install jupyter notebook --upgrade
!jupyter nbextension enable --py widgetsnbextension
!pip install "setfit[absa]"
!pip install spacy
!pip install tokenizers>=0.20,<0.21
!pip install transformers -U
!spacy download en_core_web_lg
!spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.6 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 6.1 MB/s eta 0:00:00

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

# New Section

In [ ]:
import warnings
from tqdm import TqdmWarning
warnings.filterwarnings('ignore', category=TqdmWarning)

from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from setfit import AbsaModel, AbsaTrainer
from transformers import TrainingArguments, EarlyStoppingCallback, AutoTokenizer
from setfit import AbsaModel, AbsaTrainer, TrainingArguments
from datasets import load_dataset
from transformers import EarlyStoppingCallback
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from setfit import AbsaModel, AbsaTrainer
from transformers import TrainingArguments

# Load the fine-tuned model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Load a sample dataset
dataset = load_dataset("glue", "mrpc")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, padding="max_length")

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# Try training
try:
    trainer.train()
except Exception as e:
    print(f"An error occurred: {e}")
    # Print additional debugging information
    print("Dataset info:", tokenized_datasets)
    print("Model config:", model.config)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("your-model-name")
tokenized_datasets = datasets.map(lambda examples: tokenizer(examples["text"], truncation=True, padding="max_length"), batched=True)

In [ ]:
model = AbsaModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    spacy_model="en_core_web_sm"
)
model = AbsaModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/all-mpnet-base-v2",
    spacy_model="en_core_web_sm"
)

In [ ]:
# Third cell: Training setup and execution
# Create sample data - replace with your actual data
train_data = pd.DataFrame({
    'text': ['The food was great', 'Service was poor'],
    'span': ['food', 'service'],
    'polarity': [1, -1]
})

# Convert DataFrame to Dataset
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_data)

# Define training arguments with correct parameters
args = TrainingArguments(
    output_dir="models",
    fp16=True,  # Replace use_amp with fp16
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Initialize trainer
trainer = AbsaTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset  # Replace with your eval data
)

# Train
trainer.train()

# Evaluate
metrics = trainer.evaluate()
print(metrics)

In [ ]:
dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")


In [ ]:
train_dataset = dataset.select(range(128))
eval_dataset = dataset.select(range(128, 256))

In [ ]:
args = TrainingArguments(
    output_dir="models",
    num_epochs=5,
    use_amp=True,
    batch_size=128,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
)


In [ ]:
from setfit import AbsaModel, AbsaTrainer
from transformers import TrainingArguments
from datasets import load_dataset
from setfit import AbsaModel

# Download from the 🤗 Hub
model = AbsaModel.from_pretrained(
    "tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-aspect",
    "tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-polarity",
)
# Run inference
preds = model("The food was great, but the venue is just way too busy.")


# Load the model
model = AbsaModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Load dataset
dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants")
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# Define training arguments
args = TrainingArguments(
    output_dir="models",
    use_amp=True,
    per_device_train_batch_size=32,  # Adjust batch size as needed
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Initialize trainer
trainer = AbsaTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train the model
trainer.train()

# Evaluate
metrics = trainer.evaluate(eval_dataset)
print(metrics)

# Save to Hub
trainer.push_to_hub("tomaarsen/setfit-absa-restaurants")


In [ ]:


splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/tomaarsen/setfit-absa-semeval-restaurants/" + splits["train"])
train_dataset = dataset.select(range(128))
eval_dataset = dataset.select(range(128, 256))
args = TrainingArguments(
    output_dir="models",
    num_train_epochs=5,
    use_amp=True,
    per_device_train_batch_size=128,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
)

In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Collecting setfit[absa]\n",
      "  Using cached setfit-1.1.0-py3-none-any.whl.metadata (12 kB)\n",
      "Collecting datasets>=2.15.0 (from setfit[absa])\n",
      "  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)\n",
      "Collecting sentence-transformers>=3 (from sentence-transformers[train]>=3->setfit[absa])\n",
      "  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)\n",
      "[notice] A new release of pip is available: 24.2 -> 24.3.1\n",
      "[notice] To update, run: python.exe -m pip install --upgrade pip\n"
     ]
    }
   ],
   "source": [
    "!pip install setfit[absa]",
    "!pip install spacy",
    "!pip install en-core-web-lg==3.7.1",
    "!python -m spacy download en_core_web_lg"
   ]
  }
 ]
}

In [ ]:
# Install necessary libraries
!pip install "setfit[absa]" spacy
!spacy download en_core_web_sm

# Ensure compatibility
!pip install tokenizers>=0.20,<0.21
!pip install transformers -U

from setfit import AbsaModel, AbsaTrainer, TrainingArguments
from datasets import load_dataset
from transformers import EarlyStoppingCallback
from setfit import AbsaModel, AbsaTrainer
from transformers import TrainingArguments
from datasets import load_dataset


In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-aspect")
dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants")
tokenized_datasets = dataset.map(lambda examples: tokenizer(examples["text"], truncation=True, padding="max_length"), batched=True)

In [ ]:

# Initialize the AbsaModel
model = AbsaModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    spacy_model="en_core_web_sm",
)

# Load the dataset
dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")
train_dataset = dataset.select(range(128))
eval_dataset = dataset.select(range(128, 256))


In [ ]:

# Set up training arguments
args = TrainingArguments(
    output_dir="models",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
    fp16=True
)

# Initialize and train the AbsaTrainer
trainer = AbsaTrainer(
    model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)
trainer.train()

# Save the trained model
trainer.save_model("absa_model")

In [ ]:
# Install necessary libraries
!pip install "setfit[absa]" spacy
!spacy download en_core_web_sm

# Ensure compatibility
!pip install tokenizers>=0.20,<0.21
!pip install transformers -U

from setfit import AbsaModel, AbsaTrainer, TrainingArguments
from datasets import load_dataset
from transformers import EarlyStoppingCallback
from setfit import AbsaModel, AbsaTrainer
from transformers import TrainingArguments
from datasets import load_dataset

# Set up training arguments
# The original code attempts to use TrainingArguments.copy() which does not exist.
# We now create a new TrainingArguments object for polarity_args using the values of args.
args = TrainingArguments(
    output_dir="models",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
    fp16=True
)

# Initialize and train the AbsaTrainer
# Pass args as polarity_args so that a new TrainingArguments object is created internally.
trainer = AbsaTrainer(
    model,
    args=args,
    polarity_args=args,  # Pass args to polarity_args
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)
trainer.train()

# Save the trained model
trainer.save_model("absa_model")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')